# ML Lab 05 "solutions"

Regression.

## Imports

In [1]:
# Data and Datasets
import pandas as pd
from sklearn.datasets import load_boston

# Data processing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, quantile_transform

# Validation methods
from sklearn.model_selection import train_test_split

# Metrics package
from sklearn import metrics

# Regression metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn.tree import export_graphviz

# Utils
from sklearn.pipeline import Pipeline
import pprint
import numpy as np
from time import time

## Utility functions

### Evaluation Function

Introduced this function, to save on repetitive code. In summary, what this function does:

* Takes as input arguments: a dataset (X and y), name of a model, a model object, and number of times to run hold-out evaluation
* It then does hold-out validation, for each run, training and testing the model
* It finally returns three arrays with the performance metrics from each hold-out validation run
  - Mean Squared Error
  - Mean Absolute Error
  - Maximum Error

In [2]:
def validate_model(X, y, model, num_runs=1000, test_size=0.2):
    """
    Performs hold-out validation of a given model on the dataset provided (X, y).
    The default number of runs is 1000, and the default training/test split is 80/20.
    """
    
    # arrays for storing performance metrics
    mse_list = np.array([])
    mae_list = np.array([])
    me_list = np.array([])
    
    for x in range(num_runs):
        # Hold-out validation - 80% training and 20% testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None)

        # Training and testing the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Appending performance metrics to arrays created above
        mse_list = np.append(mse_list, mean_squared_error(y_test, y_pred))
        mae_list = np.append(mae_list, mean_absolute_error(y_test, y_pred))
        me_list = np.append(me_list, max_error(y_test, y_pred))
    
    return mse_list, mae_list, me_list

### Result function

To save and present results in a nice manner, the following code sets up a DataFrame for results and a function to add data to it, which we'll use further below when we're testing the different regression models. 

In [3]:
result_df = pd.DataFrame(columns=['Model','MSE','MAE','ME'])

def add_result(model_name, mse_list, mae_list, me_list):
    """
    Add a result row to a pandas dataframe (created above) for a model with name (model_name).
    mse_list, mae_list and me_list are array lists of performance results from different hold-out validation runs,
    coming from the validate_model() function, above.
    """
    global result_df # doing this to be able to access the gloval dataframe defined above the function
    
    new_row = {'Model':model_name, 'MSE':mse_list.mean(), 'MAE':mae_list.mean(), 'ME':me_list.mean()}
    result_df = result_df.append(new_row, ignore_index=True)
    
    return result_df

### Default variables

In [4]:
# sets the number of runs of hold-out validation below (from which we get mean and stdev statistics)
num_runs = 100

## Loading dataset

In [5]:
# Loading the dataset
dataset = load_boston()

print(dataset['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [6]:
# Convert to Pandas DataFrame
boston_df = pd.DataFrame(dataset.data,columns=dataset.feature_names)
boston_df['target'] = dataset.target  # Median value of owner-occupied homes in $1000's

# Storing references to feature names
features = dataset['feature_names']

# Get the X (feature matrix) and y (target vector) from the data
X, y = dataset.data, dataset.target

# And just to ensure we've loaded what we expect..
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Linear Regression vs Polynomial Regression vs CART

Linear Regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Polynomial Regression: https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions

CART: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

Metrics: https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics

### Linear Regression

In [7]:
# Instantiate model
lireg_model = LinearRegression()

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, lireg_model, num_runs)

# save the result
add_result('Linear Regression', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  24.13 (+/- 12.58)
MAE:  3.41  (+/- 0.64)
ME:   20.86 (+/- 11.26)


### Polynomial Regression

Doing 1st, 2nd and 3rd order polynomials here, in each respective cell below.

In [8]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=1)),
                       ('linear', LinearRegression())])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (1st Order)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  22.99 (+/- 9.69)
MAE:  3.36  (+/- 0.54)
ME:   20.25 (+/- 11.13)


In [9]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('linear', LinearRegression())])
#('linear', LinearRegression(fit_intercept=False))])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (2nd Order)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  31.59 (+/- 120.09)
MAE:  3.36  (+/- 3.32)
ME:   23.99 (+/- 32.22)


In [10]:
# Instantiate model
poly_model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                       ('linear', LinearRegression())])
#('linear', LinearRegression(fit_intercept=False))])

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model, num_runs)

# save the result
add_result('Polynomial Regression (3rd Order)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  961922.86 (+/- 8131033.23)
MAE:  230.54  (+/- 319.88)
ME:   4870.26 (+/- 14153.84)


### CART

In [11]:
# Instantiate model
cart_model = DecisionTreeRegressor()

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, cart_model, num_runs)

# save the result
add_result('CART', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  20.95 (+/- 17.09)
MAE:  2.99  (+/- 0.70)
ME:   21.06 (+/- 16.42)


### Results

Which model appears to be best? and, is it best according to all metrics?

In a test run, one model was best on all metrics. But for 2nd and 3rd, it depended on the metric.

In [12]:
result_df

,Model,MSE,MAE,ME
0,Linear Regression,24.125089,3.406108,20.859647
1,Polynomial Regression (1st Order),22.991484,3.361561,20.250564
2,Polynomial Regression (2nd Order),31.590153,3.364170,23.994600
3,Polynomial Regression (3rd Order),961922.858771,230.541461,4870.261883
4,CART,20.952360,2.989598,21.062000


## Data transformation of target

Using a quantile transformer here, based on [[this](https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html#sphx-glr-auto-examples-compose-plot-transformed-target-py)] article from Scikit-Learn.

### Training and testing the models

In [13]:
# Instantiate model
lireg_model_trans = TransformedTargetRegressor(
    regressor=LinearRegression(),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, lireg_model_trans, num_runs)

# save the result
add_result('Linear Regression (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  27.16 (+/- 11.88)
MAE:  3.45  (+/- 0.63)
ME:   23.51 (+/- 9.70)


In [14]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=1)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (1st Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  27.47 (+/- 14.74)
MAE:  3.52  (+/- 0.78)
ME:   22.41 (+/- 10.25)


In [15]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (2nd Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  39.46 (+/- 97.16)
MAE:  3.86  (+/- 3.74)
ME:   25.44 (+/- 13.45)


In [16]:
# Instantiate model
poly_model_trans = TransformedTargetRegressor(
    regressor=Pipeline([('poly', PolynomialFeatures(degree=3)),
                       ('linear', LinearRegression())]),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, poly_model_trans, num_runs)

# save the result
add_result('Polynomial Regression (3rd Order) (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  484.01 (+/- 124.38)
MAE:  18.98  (+/- 3.35)
ME:   44.26 (+/- 3.35)


In [17]:
# Instantiate model
cart_model_trans = TransformedTargetRegressor(
    regressor=DecisionTreeRegressor(),
    transformer=QuantileTransformer(n_quantiles=200,
                                    output_distribution='normal'))

# arrays for storing performance metrics
mse_list, mae_list, me_list = validate_model(X, y, cart_model_trans, num_runs)

# save the result
add_result('CART (trans)', mse_list, mae_list, me_list)

# printing out the performance metrics (mean + standard deviation)
print("MSE:  %0.2f (+/- %0.2f)" % (mse_list.mean(), mse_list.std() * 2))
print("MAE:  %0.2f  (+/- %0.2f)" % (mae_list.mean(), mae_list.std() * 2))
print("ME:   %0.2f (+/- %0.2f)" % (me_list.mean(), me_list.std() * 2))

MSE:  18.30 (+/- 12.09)
MAE:  2.95  (+/- 0.63)
ME:   17.84 (+/- 12.37)


### Results

Did the performance of any of the models improve when transforming the target values?

In [18]:
result_df

,Model,MSE,MAE,ME
0,Linear Regression,24.125089,3.406108,20.859647
1,Polynomial Regression (1st Order),22.991484,3.361561,20.250564
2,Polynomial Regression (2nd Order),31.590153,3.364170,23.994600
3,Polynomial Regression (3rd Order),961922.858771,230.541461,4870.261883
4,CART,20.952360,2.989598,21.062000
5,Linear Regression (trans),27.164105,3.446642,23.509452
6,Polynomial Regression (1st Order) (trans),27.474923,3.517479,22.412078
7,Polynomial Regression (2nd Order) (trans),39.458678,3.860435,25.442411
8,Polynomial Regression (3rd Order) (trans),484.014130,18.977420,44.260893
9,CART (trans),18.298357,2.950196,17.843000
